In [ ]:
%matplotlib inline



MLFM-AG Model Fitting
=====================

.. currentmodule:: pydygp.linlatentforcemodels

This note decribes how to carry out the porcess of carrying out MAP parameter
estimation for the MLFM model using the Adaptive Gradient matching approximation
:class:`MLFMAdapGrad` 



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pydygp.linlatentforcemodels import MLFMAdapGrad

Model Setup
~~~~~~~~~~~
We are going to demonstrate using the example we have already described in
`tutorials-mlfm-sim` and the steps described there for setting up
this model and simulating some data. The model was a simple ODE on the unit
circle

\begin{align}S^1 = \{ x \in \mathbb{R}^2 \; : \; \| x \| = 1 \},\end{align}

We can write a linear ode on $S^1$ as

\begin{align}\begin{bmatrix} \dot{x}(t) \\ \dot{y}(t) \end{bmatrix}
   = \begin{bmatrix} 0 & -1 \\ 1 & 0 \end{bmatrix}
     \begin{bmatrix} x(t) \\ y(t) \end{bmatrix}.\end{align}

We can set up this model by and simulate some observations by




In [ ]:
import pydygp.liealgebras

np.random.seed(7)  # seed for reproducability

so2 = pydygp.liealgebras.so(2)  # import the Lie algebra
mlfm = MLFMAdapGrad(so2, R=1)   # Build the model with 0 offset matrix

tt = np.linspace(0., 5., 11)    # time data points
Y, gtrue = mlfm.sim([1., 0], tt)    # Simulation result

Model Fit
~~~~~~~~~
The API is designed to be as simple as possible and so for simple enough
models we should get reasonable results by simply calling



In [ ]:
res0 = mlfm.fit(tt, Y)

this returns a simple :class:`namedtuple` object which has attributes with
names corresponding to the parameters described in
`tutorials-index-mlfmag-par`. We can compare the point estimates with
the true latent force functions



In [ ]:
ttdense = np.linspace(tt[0], tt[-1], 21)  # make a set of dense times
fig0, ax = plt.subplots()
ax.plot(ttdense, gtrue[0](ttdense), 'k-', alpha=0.5)
ax.plot(tt, res0.g.T, '+')

Sample Density Dependenance
~~~~~~~~~~~~~~~~~~~~~~~~~~~
The adaptive gradient matching processes depend on the use of GP
interpolators of the unknown state trajectories, and therefore
we might expect the accuracy of these methods to decrease not with the
sample size but with the space between samples.

Dense Predictions
~~~~~~~~~~~~~~~~~



In [ ]:
mlfm2 = MLFMAdapGrad(so2, R=1)
data_inds = np.linspace(0, ttdense.size-1, tt.size, dtype=np.intp)
mlfm2._setup_times(tt, tt_aug=ttdense, data_inds=data_inds)
res_dense = mlfm2.fit(tt, Y, logpsi_is_fixed=True)

fig, ax = plt.subplots()
ss = np.linspace(tt[0], tt[-1])
ax.plot(ss, gtrue[0](ss), 'k-', alpha=0.2)
ax.plot(mlfm.ttc, res0.g.T, '+')
ax.plot(mlfm2.ttc, res_dense.g.T, 'o')

plt.show()